# DATA 200 Final Project (Peng Gu, Xi Chen, Kewei Sui)

In [4]:
# import all packages
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "whitegrid", 
        color_codes = True,
        font_scale = 1.5)

In [5]:
# import all datasets
df_college = pd.read_csv('dataset/college.csv')
df_player = pd.read_csv('dataset/2012-18_playerBoxScore.csv')
df_player['gmDate'] = pd.to_datetime(df_player['gmDate'], format='%Y-%m-%d')
df_player.head()

,gmDate,gmTime,seasTyp,playLNm,playFNm,teamAbbr,teamConf,teamDiv,teamLoc,teamRslt,...,playFT%,playORB,playDRB,playTRB,opptAbbr,opptConf,opptDiv,opptLoc,opptRslt,opptDayOff
0,2012-10-30,19:00,Regular,Price,A.J.,WAS,East,Southeast,Away,Loss,...,1.0,1,1,2,CLE,East,Central,Home,Win,0
1,2012-10-30,19:00,Regular,Ariza,Trevor,WAS,East,Southeast,Away,Loss,...,0.5,1,2,3,CLE,East,Central,Home,Win,0
2,2012-10-30,19:00,Regular,Okafor,Emeka,WAS,East,Southeast,Away,Loss,...,0.5,5,2,7,CLE,East,Central,Home,Win,0
3,2012-10-30,19:00,Regular,Beal,Bradley,WAS,East,Southeast,Away,Loss,...,1.0,0,3,3,CLE,East,Central,Home,Win,0
4,2012-10-30,19:00,Regular,Booker,Trevor,WAS,East,Southeast,Away,Loss,...,0.0,1,0,1,CLE,East,Central,Home,Win,0


In [13]:
start_dates = ['2012-10-30', '2013-10-29', '2014-10-28', '2015-10-27','2016-10-25','2017-10-17','2018-10-16']
for i in range(len(start_dates)):
    start_dates[i] = datetime.datetime.strptime(start_dates[i],'%Y-%m-%d')
# print(start_dates)
def setSeasons(gm_date):
    for i in range(len(start_dates) - 1):
        if gm_date >= start_dates[i] and gm_date < start_dates[i + 1]:
            return start_dates[i].year
df_player["seasYear"] = df_player['gmDate'].apply(lambda x:setSeasons(x))
df_player.tail(10)

,gmDate,gmTime,seasTyp,playLNm,playFNm,teamAbbr,teamConf,teamDiv,teamLoc,teamRslt,...,playORB,playDRB,playTRB,opptAbbr,opptConf,opptDiv,opptLoc,opptRslt,opptDayOff,seasYear
155703,2018-04-11,10:30,Regular,Onuaku,Chinanu,HOU,West,Southwest,Away,Loss,...,1,3,4,SAC,West,Pacific,Home,Win,2,2017
155704,2018-04-11,10:30,Regular,Bogdanovic,Bogdan,SAC,West,Pacific,Home,Win,...,0,5,5,HOU,West,Southwest,Away,Loss,1,2017
155705,2018-04-11,10:30,Regular,Jackson,Justin,SAC,West,Pacific,Home,Win,...,0,4,4,HOU,West,Southwest,Away,Loss,1,2017
155706,2018-04-11,10:30,Regular,Cauley-Stein,Willie,SAC,West,Pacific,Home,Win,...,1,10,11,HOU,West,Southwest,Away,Loss,1,2017
155707,2018-04-11,10:30,Regular,Fox,De'Aaron,SAC,West,Pacific,Home,Win,...,1,7,8,HOU,West,Southwest,Away,Loss,1,2017
155708,2018-04-11,10:30,Regular,Sampson,JaKarr,SAC,West,Pacific,Home,Win,...,0,4,4,HOU,West,Southwest,Away,Loss,1,2017
155709,2018-04-11,10:30,Regular,Hield,Chavano,SAC,West,Pacific,Home,Win,...,2,3,5,HOU,West,Southwest,Away,Loss,1,2017
155710,2018-04-11,10:30,Regular,Hayes,Nigel,SAC,West,Pacific,Home,Win,...,1,3,4,HOU,West,Southwest,Away,Loss,1,2017
155711,2018-04-11,10:30,Regular,Carter,Vincent,SAC,West,Pacific,Home,Win,...,0,3,3,HOU,West,Southwest,Away,Loss,1,2017
155712,2018-04-11,10:30,Regular,Cooley,Jack,SAC,West,Pacific,Home,Win,...,1,3,4,HOU,West,Southwest,Away,Loss,1,2017
